In [15]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
import datasets
import sentence_transformers
from sentence_transformers import SentenceTransformer


In [33]:
train=datasets.load_from_disk('D:\\SBERT-Training\\datasets\\SNLI\\train')
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 550152
})

In [34]:
test=datasets.load_from_disk('D:\\SBERT-Training\\datasets\\SNLI\\test')
test

FileNotFoundError: Directory D:\SBERT-Training\datasets\SNLI\test not found

In [35]:

# Load the dataset
train_df=pd.DataFrame({'premise':train['premise'],
                         'hypothesis':train['hypothesis'],
                         'label':train['label']})
train_df


,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
...,...,...,...
550147,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',2
550148,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",1
550149,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,1
550150,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,2


In [36]:
test_df=pd.DataFrame({'premise':test['premise'],
                         'hypothesis':test['hypothesis'],
                         'label':test['label']})
test_df



,premise,hypothesis,label
0,In the event that a receiving party or its Ass...,Receiving Party shall not reverse engineer any...,2
1,The Recipient shall immediately return and red...,Receiving Party shall destroy or return some C...,1
2,"No other right or license, whether expressed o...",Agreement shall not grant Receiving Party any ...,1
3,In the event that a receiving party or its Ass...,Receiving Party shall not disclose the fact th...,2
4,"For purposes of this Agreement, ""Confidential ...",Confidential Information shall only include te...,0
...,...,...,...
1986,"2. As used herein, “Confidential Information” ...",Receiving Party shall not solicit some of Disc...,2
1987,9. Upon the Disclosing Party’s written request...,Receiving Party may create a copy of some Conf...,2
1988,The Recipient agrees that if the Recipient is ...,Receiving Party shall notify Disclosing Party ...,1
1989,1.2 Notwithstanding anything herein to the con...,Receiving Party may acquire information simila...,1


In [16]:
path='D:\\distilbert-base-uncased'
model=SentenceTransformer(path)

No sentence-transformers model found with name D:\distilbert-base-uncased. Creating a new one with mean pooling.


In [18]:

# Load the pre-trained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\aakha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aakha\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
len(train_df['premise'][:100])

100

In [41]:

# Preprocess the data
train_encodings = tokenizer(train_df['premise'][:500].to_list(), train_df['hypothesis'][:500].to_list(), 
                             return_tensors='pt', 
                             max_length=512, 
                             padding='max_length', 
                             truncation=True)


In [42]:

test_encodings = tokenizer(test_df['premise'][:500].to_list(), test_df['hypothesis'][:500].to_list(), 
                            return_tensors='pt', 
                            max_length=512, 
                            padding='max_length', 
                            truncation=True)


In [43]:

# Create labels
train_labels = train_df['label']
test_labels = test_df['label']


In [44]:
import torch

In [45]:

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(**train_encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    train_accuracy = accuracy_score(train_labels, predictions.cpu().numpy())

    outputs = model(**test_encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    test_accuracy = accuracy_score(test_labels, predictions.cpu().numpy())


In [31]:

print("Multinli Train Accuracy:", train_accuracy)
print("Multinli Test Accuracy:", test_accuracy)

NameError: name 'train_accuracy' is not defined